# Attention is all your need

## 概念补充
### 位置编码
- 位置编码（PE）：由于 Transformer 不像 RNN 那样具有时间序列性质，需通过位置编码（Positional Encoding）保留输入序列的顺序信息。
    - 使用正弦和余弦函数为每一个位置生成唯一编码：
        - 公式为：$$\begin{aligned}
    P E_{(p o s, 2 i)} & =\sin \left(\frac{p o s}{10000^{2 i / d_{\text {model }}}}\right), \\
    P E_{(p o s, 2 i+1)} & =\cos \left(\frac{p o s}{10000^{2 i / d_{\text {model }}}}\right) .
    \end{aligned}$$其中pos是位置索引，i是维度索引，$d_{model}$是嵌入向量的维度

## 正则化方法
- **Dropout**：对每个子层的输出进行随机丢弃（例如，Add & Nrom）
- **平滑标签(Label Smoothing)**：
    - 硬标签（hard labels）我们理解的标签属于这个范畴。本质上是一种 one-hot（[0,0,1,0,...,0]）
    - 软标签（soft labels）由于硬标签对于模型训练比较极端（softmax只有 logits $\rightarrow \infty$ 才逼近1）
    - **概念**：正确类别稍小于1（$1 - \epsilon_{ls}$），其他稍大于0 （所有类别平分 $\epsilon_{ls}$ ）。
        - 具体来说 $$ y = [0, 0, ..., 1, ..., 0] \rightarrow y' = [\frac{\epsilon_{ls}}{C}, \frac{\epsilon_{ls}}{C}, ... , 1 - \epsilon_{ls} + \frac{\epsilon_{ls}}{C}, ..., \frac{\epsilon_{ls}}{C} ] $$

```python 
smooth = (1 - epsilon) * one_hot + epsilon / C
```



## BLEU 分数
- BLEU（Bilingual Evaluation Understudy） 双语评估替换
- 公式：
    $$
    BLEU = BP \cdot \exp\left(\sum_{n=1}^{N} w_n \log p_n\right)
    $$
    其中 $BP$ 是 brevity penalty（简短惩罚），$N$ 是 n-gram 的最大长度（通常为 4），$w_n$ 是权重（通常取均匀权重 $w_n = \frac{1}{N}$），$p_n$ 是 n-gram 精度


- 首先要明确两个概念：
    - N-gram：用来描述句子中的一组 n 个连续的单词。比如，"Thank you so much" 中的 n-grams：
        - 1-gram: "Thank", "you", "so", "much"
        - 2-gram: "Thank you", "you so", "so much"
        - 3-gram: "Thank you so", "you so much"
        - 4-gram: "Thank you so much" 需要注意的一点是，n-gram 中的单词是按顺序排列的，所以 "so much Thank you" 不是一个有效的 4-gram。



##  Brevity Penalty

这里引出了 **brevity penalty**，这是一个惩罚因子，公式如下：

$$BP = 
\begin{cases} 
1 & \text{if } c > r \\ 
e^{1 - \frac{r}{c}} & \text{if } c \leq r 
\end{cases}$$

其中 $c$ 是 candidate 的长度，$r$ 是 reference 的长度。

当候选译文的长度 $c$ 等于参考译文的长度 $r$ 的时候，$BP = 1$，当候选翻译的文本长度较短的时候，用 $e^{1 - \frac{r}{c}}$ 作为 BP 值。

回到原来的公式：$$BLEU = BP \cdot \exp\left(\sum_{n=1}^{N} w_n \log p_n\right)$$，汇总一下符号定义：

- $BP$ 文本长度的惩罚因子
- $N$ n-gram 中 $n$ 的最大值
- $w_n$ 权重
- $p_n$ n-gram 的精度 (precision)

# Attention scores
给定查询矩阵 $Q$、键矩阵 $K$ 和值矩阵 $V$，其注意力输出的数学表达式如下：

$$Attention(Q, K, V) = Softmax\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

- $Q$ (Query)：用于查询的向量矩阵。
- $K$ (Key)：表示键的向量矩阵，用于与查询匹配。
- $V$ (Value)：值矩阵，注意力权重最终会作用在该矩阵上。
- $d_k$：键或查询向量的维度。

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
torch.__version__

'2.5.1+cu118'

缩放点积注意力

In [3]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    缩放点积注意力计算。
    
    参数:
        Q: 查询矩阵 (batch_size, seq_len_q, embed_size)
        K: 键矩阵 (batch_size, seq_len_k, embed_size)
        V: 值矩阵 (batch_size, seq_len_v, embed_size)
        mask: 掩码矩阵，用于屏蔽不应该关注的位置 (可选)

    返回:
        output: 注意力加权后的输出矩阵
        attention_weights: 注意力权重矩阵
    """
    # get d_k
    embed_size = Q.size(-1)

    # compute product / importance! divide sqrt(d_k)
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(embed_size)

    # 如果是带掩码的attention计算，掩码设置为负无穷
    if mask is not None:
        scores = scores.masked_fill(mask==0, float('-inf'))

    # 在最后一维计算注意力分数, key value to probs
    # seq_len_k == seq_len_v 
    attention_weight = F.softmax(scores, dim=-1)
    output = torch.matmul(attention_weight, V) 
     
    return output, attention_weight

- mask
    - padding：忽略填充（对齐序列长度）的位置
    - look-ahead mask：防止偷看正确答案(TransformerBlock 里面的 Masked Attention)

In [4]:
Q = torch.arange(12, dtype=torch.float32).view(1, 3, 4)
K = torch.arange(16, dtype=torch.float32).view(1, 4, 4 )
V = K
print("Q:",Q,"\nK", K,"\nV", V)
output, attention_weight = scaled_dot_product_attention(Q,K,V)
print("output:", output, "\nattention_weight:",attention_weight)

Q: tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]]) 
K tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]]]) 
V tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]]])
output: tensor([[[12.0000, 13.0000, 14.0000, 15.0000],
         [12.0000, 13.0000, 14.0000, 15.0000],
         [12.0000, 13.0000, 14.0000, 15.0000]]]) 
attention_weight: tensor([[[2.3195e-16, 3.7751e-11, 6.1442e-06, 9.9999e-01],
         [0.0000e+00, 6.0546e-39, 7.7811e-20, 1.0000e+00],
         [0.0000e+00, 0.0000e+00, 9.8542e-34, 1.0000e+00]]])


## 单头注意力以及自注意力

In [18]:
class SingleHeadAttention(nn.Module):
    def __init__(self, embed_size):
        """
        单头注意力机制。
        
        参数:
            embed_size: 输入序列（Inputs）的嵌入（Input Embedding）维度，也是论文中所提到的d_model。
        """
        super(SingleHeadAttention, self).__init__()
        self.W_q = nn.Linear(embed_size, embed_size)
        self.W_k = nn.Linear(embed_size, embed_size)
        self.W_v = nn.Linear(embed_size, embed_size)

    def forward(self, q, k, v, mask=None):
        """
        前向传播，
        输入：
            qkv矩阵，以及mask
        返回：
            注意力分加权的输出，注意力权重
        """
        Q = self.W_q(q)
        K = self.W_k(k)
        V = self.W_v(v)
        output, attention_weight = scaled_dot_product_attention(Q, K, V, mask)
        return output, attention_weight


继续忽视多头（multi-head）观察输入，线条一分为三传入 Attention 模块，这意味着查询（query）、键（key）和值（value）实际上都来自同一输入序列**x**
![selfattention](./img/Self-attention.png)
```python
# 定义线性层，用于生成查询、键和值矩阵
self.w_q = nn.Linear(embed_size, embed_size)
self.w_k = nn.Linear(embed_size, embed_size)
self.w_v = nn.Linear(embed_size, embed_size)
```

In [7]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size):
        """
        自注意力（Self-Attention）机制。
        
        参数:
            embed_size: 输入序列的嵌入维度（每个向量的特征维度）。
        """
        super(SelfAttention, self).__init__()
        self.attention = SingleHeadAttention(embed_size)  # 使用通用Attention模块

    def forward(self, x, mask=None):
        """
        前向传播函数。
        
        参数:
            x: 输入序列 (batch_size, seq_len, embed_size)
            mask: 掩码矩阵 (batch_size, seq_len, seq_len)

        返回:
            out: 自注意力加权后的输出 (batch_size, seq_len, embed_size)
            attention_weights: 注意力权重矩阵 (batch_size, seq_len, seq_len)
        """
        # 在自注意力机制中，q, k, v 都来自同一输入序列
        # q = k = v = x
        out, attention_weights = self.attention(x, x, x, mask)

        return out, attention_weights


## 交叉注意力

![](./img/CrossAttention.png)
- block的顺序是 self_attention $\rightarrow$ cross_attention
- 也即从encoder获取kv的信息
    - 数学表达式：
    $$Q = X_{\text{decoder}} W^Q, \quad K = X_{\text{encoder}} W^K, \quad V = X_{\text{encoder}} W^V$$

In [10]:
class CrossAttention(nn.Module):
    def __init__(self, embed_size):
        """"
        交叉注意力
        参数：
            embed_size：输入序列的嵌入维度
        """
        super(CrossAttention,self).__init__()
        self.attention = SingleHeadAttention(embed_size)
    def forward(self, q, kv, mask=None):
        """
        前向传播函数。
        
        参数:
            query: 查询矩阵的输入 (batch_size, seq_len_q, embed_size)
            kv: 键和值矩阵的输入 (batch_size, seq_len_kv, embed_size)
            mask: 掩码矩阵 (batch_size, seq_len_q, seq_len_kv)

        返回:
            out: 注意力加权后的输出 (batch_size, seq_len_q, embed_size)
            attention_weights: 注意力权重矩阵 (batch_size, seq_len_kv, seq_len_kv)
        """
        # 根据数学表达式，q来自decoder，kv来自encoder
        output, attention_weight = self.attention(q, kv, kv, mask)

        return output, attention_weight

- **注意这里的 $q \neq k = v$**

## MHA

假设我们有 $h$ 个头，每个头拥有独立的线性变换矩阵 $W_i^Q, W_i^K, W_i^V$ (分别作用于查询、键和值的映射)，每个头的计算如下：
$$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$$
这些头的输出将沿最后一维拼接（Concat），并通过线性变换矩阵  $W^O$  映射回原始嵌入维度（embed_size）：
$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O$$
其中 $h$ 是head_num

先从符合直觉的角度构造多头。

In [12]:
embed_size = 64
num_heads = 3
w_q = nn.ModuleList([nn.Linear(embed_size, embed_size) for _ in range(num_heads)])
w_q

ModuleList(
  (0-2): 3 x Linear(in_features=64, out_features=64, bias=True)
)

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads

        # 给每一个head单独定义QKV的线性层，输出维度为embed_size
        self.w_q = nn.ModuleList(
            [nn.Linear(embed_size, embed_size) for _ in range(num_heads)]
        )
        self.w_k = nn.ModuleList(
            [nn.Linear(embed_size, embed_size) for _ in range(num_heads)]
        )
        self.w_v = nn.ModuleList(
            [nn.Linear(embed_size, embed_size) for _ in range(num_heads)]
        )

        # 输出线性层，用于将多头拼接后的输出映射回 embed_size
        self.fc_out = nn.Linear(num_heads * embed_size, embed_size)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        缩放点积注意力计算。
        
        参数:
            Q: 查询矩阵 (batch_size, seq_len_q, embed_size)
            K: 键矩阵 (batch_size, seq_len_k, embed_size)
            V: 值矩阵 (batch_size, seq_len_v, embed_size)
            mask: 掩码矩阵，用于屏蔽不应该关注的位置 (可选)

        返回:
            output: 注意力加权后的输出矩阵
            attention_weights: 注意力权重矩阵
        """
        # get d_k
        embed_size = Q.size(-1)

        # compute product / importance! divide sqrt(d_k)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(embed_size)

        # 如果是带掩码的attention计算，掩码设置为负无穷
        if mask is not None:
            scores = scores.masked_fill(mask==0, float('-inf'))

        # 在最后一维计算注意力分数, key value to probs
        # seq_len_k == seq_len_v 
        attention_weight = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weight, V) 
        
        return output, attention_weight 

    def forward(self, q, k, v, mask=None):
        """
        前向传播函数。
        
        参数:
            q: 查询矩阵 (batch_size, seq_len_q, embed_size)
            k: 键矩阵 (batch_size, seq_len_k, embed_size)
            v: 值矩阵 (batch_size, seq_len_v, embed_size)
            mask: 掩码矩阵 (batch_size, seq_len_q, seq_len_k)

        返回:
            out: 注意力加权后的输出
            attention_weights: 注意力权重矩阵
        """
        batch_size = q.shape[0]
        multi_head_output = []
        multi_head_weights = []
        for i in range(self.num_heads):
            Q = self.w_q[i](q)
            K = self.w_k[i](k)
            V = self.w_v[i](v)

            # 计算注意力分数
            scores, attn_weights = scaled_dot_product_attention(Q, K, V, mask)
            multi_head_output.append(scores)
            multi_head_weights.append(attn_weights)

        # (B, S_q, embed_size) --concate-->  (B, S_q, num_heads * embed_size)
        concat_out = torch.cat(multi_head_output, dim=-1)
        output = self.fc_out(concat_out)
        return output

In [22]:
MultiHeadAttention(512,1),SingleHeadAttention(512),MultiHeadAttention(512,8)

(MultiHeadAttention(
   (w_q): ModuleList(
     (0): Linear(in_features=512, out_features=512, bias=True)
   )
   (w_k): ModuleList(
     (0): Linear(in_features=512, out_features=512, bias=True)
   )
   (w_v): ModuleList(
     (0): Linear(in_features=512, out_features=512, bias=True)
   )
   (fc_out): Linear(in_features=512, out_features=512, bias=True)
 ),
 SingleHeadAttention(
   (W_q): Linear(in_features=512, out_features=512, bias=True)
   (W_k): Linear(in_features=512, out_features=512, bias=True)
   (W_v): Linear(in_features=512, out_features=512, bias=True)
 ),
 MultiHeadAttention(
   (w_q): ModuleList(
     (0-7): 8 x Linear(in_features=512, out_features=512, bias=True)
   )
   (w_k): ModuleList(
     (0-7): 8 x Linear(in_features=512, out_features=512, bias=True)
   )
   (w_v): ModuleList(
     (0-7): 8 x Linear(in_features=512, out_features=512, bias=True)
   )
   (fc_out): Linear(in_features=4096, out_features=512, bias=True)
 ))

Q: 现在所说的性能“提升”真的是由多头造成的吗？

![](./img/thinking%20deepth.png)

**实际上这种构造的方式使得参数量线性增长，并且在信息的拟合上可能存在overfit（每一个qkv的参数都是全空间的参数）**

### True MHA 1

In [ ]:
class MHA(nn.Module):
    def __init__(self, embed_size, num_heads):
        """"
        MultiHeadAttention 
        args:
            embed_size: 输入序列的嵌入维度
            num_heads: 注意力头的数量
            注意，head_dim = embed_size / num_heads(必须能够整除)
        """
        super(MHA, self).__init__()
        assert embed_size % num_heads == 0 , "embed_size must divide by num_heads"

        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads


        # 给每一个head单独定义QKV的线性层，输出维度为embed_size
        self.w_q = nn.ModuleList(
            [nn.Linear(embed_size, self.head_dim) for _ in range(num_heads)]
        )
        self.w_k = nn.ModuleList(
            [nn.Linear(embed_size, self.head_dim) for _ in range(num_heads)]
        )
        self.w_v = nn.ModuleList(
            [nn.Linear(embed_size, self.head_dim) for _ in range(num_heads)]
        )

        # 输出线性层，用于将多头拼接后的输出映射回 embed_size
        self.fc_out = nn.Linear(num_heads * self.head_dim, embed_size)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        缩放点积注意力计算。
        
        参数:
            Q: 查询矩阵 (batch_size, seq_len_q, embed_size)
            K: 键矩阵 (batch_size, seq_len_k, embed_size)
            V: 值矩阵 (batch_size, seq_len_v, embed_size)
            mask: 掩码矩阵，用于屏蔽不应该关注的位置 (可选)

        返回:
            output: 注意力加权后的输出矩阵
            attention_weights: 注意力权重矩阵
        """
        # get d_k
        embed_size = Q.size(-1)

        # compute product / importance! divide sqrt(d_k)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(embed_size)

        # 如果是带掩码的attention计算，掩码设置为负无穷
        if mask is not None:
            scores = scores.masked_fill(mask==0, float('-inf'))

        # 在最后一维计算注意力分数, key value to probs
        # seq_len_k == seq_len_v 
        attention_weight = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weight, V) 
        
        return output, attention_weight 

    def forward(self, q, k, v, mask=None):
        """
        前向传播函数。
        
        参数:
            q: 查询矩阵 (batch_size, seq_len_q, head_dim)
            k: 键矩阵 (batch_size, seq_len_k, head_dim)
            v: 值矩阵 (batch_size, seq_len_v, head_dim)
            mask: 掩码矩阵 (batch_size, seq_len_q, seq_len_k)

        返回:
            out: 注意力加权后的输出
            attention_weights: 注意力权重矩阵
        """
        batch_size = q.shape[0]
        multi_head_output = []
        multi_head_weights = []
        for i in range(self.num_heads):
            Q = self.w_q[i](q)
            K = self.w_k[i](k)
            V = self.w_v[i](v)

            # 计算注意力分数
            scores, attn_weights = scaled_dot_product_attention(Q, K, V, mask)
            multi_head_output.append(scores)
            multi_head_weights.append(attn_weights)

        # (B, S_q, head_dim) --concate-->  (B, S_q, num_heads * head_dim)
        concat_out = torch.cat(multi_head_output, dim=-1)
        output = self.fc_out(concat_out)
        return output    
        

In [29]:
MHA(512, 8)

MHA(
  (w_q): ModuleList(
    (0-7): 8 x Linear(in_features=512, out_features=64, bias=True)
  )
  (w_k): ModuleList(
    (0-7): 8 x Linear(in_features=512, out_features=64, bias=True)
  )
  (w_v): ModuleList(
    (0-7): 8 x Linear(in_features=512, out_features=64, bias=True)
  )
  (fc_out): Linear(in_features=512, out_features=512, bias=True)
)

虽然逻辑上很直观，但是运行速度极慢。接下来实现tranformers版本的MHA（矩阵运算）

![](./img/MHAinAttentionIsAllYourNeed.png)

首先修改一下变量名：
- `embed_size` $\rightarrow$ $d_{model}$
- `num_heads` $\rightarrow$ h
- `head_dim` $\rightarrow$ $d_k$

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional
import math

class TransformerMHA(nn.Module):
    def __init__(self, d_model, h):
        """
        多头注意力机制
        args:
            d_model:输入序列的维度大小
            h：attention heads的数量 
        """
        super(TransformerMHA, self).__init__()
        assert d_model % h ==0, "d_model must be divided by h (number of heads)"

        self.d_model = d_model 
        self.h = h

        # QKV Linear layer
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.fc_out = nn.Linear(d_model, d_model)
    def forward(self, q, k, v, mask=None):
        """
        args: 
            i = q,k,v shape is (B, seq_len_{i}, d_model)
            mask shape is (B, 1, seq_len_q, seq_len_k)
            and seq_len_k = seq_len_v unequal to seq_len_q
        return:
            outputs:(weighted) shape is (batch_size, h, seq_len_q, seq_len_kv)
            attntion_weights shape is (batch_size, h, seq_len_q, seq_len_k)
        """
        batch_size, seq_len_q, _ = q.size()
        seq_len_kv, _ , _ = k.size()

        # 将d_model 拆分为 head_numbers and head_dim 
        # 并且 (B, S_ , num_head, head_dim)  ---> (B , num_head, , S_, head_dim)
        Q = self.w_q(q).view(batch_size, seq_len_q, self.h, -1).transpose(1,2)
        K = self.w_k(q).view(batch_size, seq_len_kv, self.h, -1).transpose(1,2)
        V = self.w_v(q).view(batch_size, seq_len_kv, self.h, -1).transpose(1,2)

        scaled_attention, _ = scaled_dot_product_attention(Q, K, V, mask=None)

        concat_out = scaled_attention.transpose(1,2).contiguous()
        concat_out.view(batch_size, -1, self.d_model)

        out = self.fc_out(concat_out)
        return out 

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        缩放点积注意力计算。
        
        args:
            Q: 查询矩阵 (batch_size, num_heads, seq_len_q, d_k)
            K: 键矩阵 (batch_size, num_heads, seq_len_kv, d_k)
            V: 值矩阵 (batch_size, num_heads, seq_len_kv, d_v)
            mask: 掩码矩阵 (batch_size, 1, seq_len_q, seq_len_kv) 或 
            (1, 1, seq_len_q, seq_len_kv) 或 
            (batch_size, h, seq_len_q, seq_len_kv)

        return:
            output: 注意力加权后的输出矩阵
            attention_weights: 注意力权重矩阵
        """
        d_k = Q.size(-1)  # d_k
        
        # 计算点积并进行缩放
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)

        # 如果提供了掩码矩阵，则将掩码对应位置的分数设为 -inf
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        # 对缩放后的分数应用 Softmax 函数，得到注意力权重
        attention_weights = F.softmax(scores, dim=-1)

        # 加权求和，计算输出
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights       

In [37]:
TransformerMHA(512, 8)

TransformerMHA(
  (w_q): Linear(in_features=512, out_features=512, bias=True)
  (w_k): Linear(in_features=512, out_features=512, bias=True)
  (w_v): Linear(in_features=512, out_features=512, bias=True)
  (fc_out): Linear(in_features=512, out_features=512, bias=True)
)